In [0]:
df = spark.sql("select * from samples.nyctaxi.trips")

In [0]:
display(df.limit(10))

In [0]:
df.printSchema()

In [0]:
df.dtypes

In [0]:
df.count()

In [0]:
from pyspark.sql.functions import expr

df = df.withColumn("different_zip", expr("CASE WHEN pickup_zip <> dropoff_zip THEN 'same_zip' ELSE 'different_zip' END"))

display(df.limit(10))

In [0]:
df.where("different_zip = 'same_zip'").createOrReplaceTempView("tmp_same_zip")



In [0]:
df_same_zip = spark.sql("select * from tmp_same_zip")
df_same_zip.display()
# on non-serverless compute you can do this, on serverless compute it's not supported
# df_same_zip.cache()

In [0]:
df_same_zip.count()

In [0]:
print(f"The view same_zip contains {df_same_zip.count()} rows.")

In [0]:
%sql

select * from tmp_same_zip

In [0]:
%sql
create or replace table same_zip_agg
as
select  MIN(tpep_pickup_datetime) as min_pickup_date,
        MAX(tpep_pickup_datetime) as max_pickup_date
from tmp_same_zip

In [0]:
%sql
select * from same_zip_agg

In [0]:
df_same_zip = df_same_zip.withColumnRenamed("different_zip", "long_distance")
df_same_zip.display()

In [0]:
df_same_zip.write.mode("overwrite").saveAsTable("same_zip2")


In [0]:
df_same_zip_new = spark.table("same_zip2")
df_same_zip_new.display()

In [0]:
# Function to rename a column in a DataFrame
def renameColumn(df, oldName, newName):
    df = df.withColumnRenamed(oldName, newName)
    return df

In [0]:
df_same_zip_new = renameColumn(df_same_zip_new, "long_distance", "different_zip")
df_same_zip_new.display()

In [0]:


# spark.sql("DROP TABLE IF EXISTS same_zip")
# spark.sql("DROP TABLE IF EXISTS same_zip2")
# spark.sql("DROP TABLE IF EXISTS same_zip_agg")